In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [12]:
# URL of the website
URL = "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile"
# Set up the Chrome driver
driver = webdriver.Chrome()
driver.get(URL)
driver.maximize_window()
time.sleep(5)    # Wait for the page to load

In [13]:
# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 5):  # There are 4 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('bihar.csv', index=False)

 #Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/patna-to-kathmandu: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF729BD9632+30946]
	(No symbol) [0x00007FF729B8E3C9]
	(No symbol) [0x00007FF729A86FDA]
	(No symbol) [0x00007FF729AD822C]
	(No symbol) [0x00007FF729AD850C]
	(No symbol) [0x00007FF729B1DCB7]
	(No symbol) [0x00007FF729AFCAAF]
	(No symbol) [0x00007FF729B1B041]
	(No symbol) [0x00007FF729AFC813]
	(No symbol) [0x00007FF729ACA6E5]
	(No symbol) [0x00007FF729ACB021]
	GetHandleVerifier [0x00007FF729D0F83D+1301229]
	GetHandleVerifier [0x00007FF729D1BDB7+1351783]
	GetHandleVerifier [0x00007FF729D12A03+1313971]
	GetHandleVerifier [0x00007FF729C0DD06+245686]
	(No symbol) [0x00007FF729B9758F]
	(No symbol) [0x00007FF729B93804]
	(No symbol) [0x00007FF729B93992]
	(No symbol) [0x00007FF729B8A3EF]
	BaseThreadInitThunk [0x00007FFF46E8257D+29]
	RtlUserThreadStart [0x00007FFF47CAAF28+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-tic

In [14]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),20:28,05h 32m,02:00,3.4,450,6 Seats available
1,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,V.I.P Travels,A/C Seater / Sleeper (2+2),21:40,06h 00m,03:40,3.0,INR 350,61 Seats available
2,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:20,06h 40m,05:00,3.7,INR 550,54 Seats available
3,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),22:40,05h 20m,04:00,2.3,405,27 Seats available
4,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...,Bihar state road transport corporation (BSRTC)...,Volvo AC Seater 2+2,16:30,16h 00m,08:30,2.7,INR 1200,4 Seats available
...,...,...,...,...,...,...,...,...,...,...
214,Lucknow to Gopalganj (Bihar),https://www.redbus.in/bus-tickets/lucknow-to-g...,Bihar state road transport corporation (BSRTC)...,Volvo AC Seater 2+2,23:00,08h 00m,07:00,3.1,595,8 Seats available
215,Lucknow to Gopalganj (Bihar),https://www.redbus.in/bus-tickets/lucknow-to-g...,Samay Shatabdi Travels Pvt Ltd,AC Sleeper (2+1),21:00,06h 45m,03:45,4.4,INR 999,21 Seats available
216,Lucknow to Gopalganj (Bihar),https://www.redbus.in/bus-tickets/lucknow-to-g...,MISHRA BANDHU BUS SERVICE,A/C Seater / Sleeper (2+1),21:30,07h 30m,05:00,3.9,INR 799,16 Seats available
217,Lucknow to Gopalganj (Bihar),https://www.redbus.in/bus-tickets/lucknow-to-g...,Maan Travels,NON AC Seater / Sleeper 2+1,23:55,10h 35m,10:30,1.6,INR 1050,41 Seats available
